In [69]:
import numpy as np
import pandas as pd
from itertools import product, permutations, combinations, combinations_with_replacement
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import ctypes,sys

In [12]:
model1="non_filter"
model2="filter"
model3="multiply"
model4="score"

hvg_version = "sct"

p_value_cutoff=0.05
score_cutoff1=0.1
score_cutoff2=0.05
expression_form="SCT"
link_signal_1 = "all"
link_signal_2 = "nonnegative"

dataversion1 = "count"
dataversion2 = "norm"

gene_peak_1 = "500kb"
gene_peak_2 = "linkpeaks"

data = "bmmc"

data_version_1 = dataversion1+"_"+data+"_"+gene_peak_2+"_"+"motifmatchr"
data_version_2 = dataversion1+"_"+data+"_"+gene_peak_2+"_"+"motifmatchr"
data_version_3 = dataversion1+"_"+data+"_"+gene_peak_2+"_"+"motifmatchr"
data_version_4 = dataversion1+"_"+data+"_"+gene_peak_2+"_"+"motifmatchr"

directory = "/home/zc354/GRN/output/predicted_network/"

pred_path_1=directory+model1+'_'+hvg_version+'_'+data_version_1+'.csv'
pred_path_2=directory+model2+'_'+hvg_version+'_'+data_version_2+'.csv'
pred_path_3=directory+model3+'_'+hvg_version+'_'+data_version_1+'.csv'
pred_path_4=directory+model4+'_'+hvg_version+'_'+data_version_2+'.csv'
#pred_path_2=directory+model2+'_'+hvg_version+'_'+data_version+'.csv'

true_path='/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv'

directed=False
save_path='/gpfs/gibbs/pi/zhao/zc354/GRN/output'

In [13]:
pred_path_1

'/home/zc354/GRN/output/predicted_network/non_filter_sct_count_bmmc_linkpeaks_motifmatchr.csv'

In [14]:
trueEdgesDF = pd.read_csv(true_path,sep = ',', header = 0, index_col = None)
predEdgesDF_1 = pd.read_csv(pred_path_1,sep = ',', header = 0, index_col = None)

In [ ]:
def gen_df(EdgesDF = predEdgesDF_1, path='/gpfs/gibbs/pi/zhao/zc354/GRN/output/TF_filename.csv'):
    #Read true netword and predicted network
    #EdgesDF = pd.read_csv(path,sep = '\t', header = 0, index_col = None)

    #Unification changes lowercase letters to uppercase letters
    EdgesDF['Gene1'] = EdgesDF['Gene1'].str.upper()
    EdgesDF['Gene2'] = EdgesDF['Gene2'].str.upper()

    #Drop index in predEdgesDF that Edgeweight=0
    if 'Edgeweight' in EdgesDF.columns.values.tolist() :
        EdgesDF.drop(EdgesDF[EdgesDF['Edgeweight'] == 0].index,inplace=True)
        EdgesDF = EdgesDF.sort_values(by=['Edgeweight'],ascending=False)

    #Drop selfEdges
    EdgesDF = EdgesDF.loc[(EdgesDF['Gene1'] != EdgesDF['Gene2'])]

    #Drop duplicates in trueEdgesDF and predEdgesDF
    EdgesDF.drop_duplicates(keep = 'first', subset=['Gene1','Gene2'], inplace=True)
    
    #Generate a column to represent Edges
    EdgesDF['Edges'] = EdgesDF['Gene1'] + "|" + EdgesDF['Gene2']
    EdgesDF['reverseEdges'] = EdgesDF['Gene2'] + "|" + EdgesDF['Gene1']
    return EdgesDF

In [ ]:
trueEdgesDF = gen_df(EdgesDF = trueEdgesDF,path = true_path)
predEdgesDF_1 = gen_df(EdgesDF = predEdgesDF_1,path = pred_path_1)

In [ ]:
true_TF = set(trueEdgesDF['Gene1'])
pred_TF = set(predEdgesDF_1['Gene1'])&set(predEdgesDF_2['Gene1'])
true_tg = set(trueEdgesDF['Gene2'])
pred_tg = set(predEdgesDF_1['Gene2'])&set(predEdgesDF_2['Gene2'])

In [ ]:
TFset_1 = set(trueEdgesDF['Gene1'])&set(predEdgesDF_1['Gene1'])

In [74]:
def Early_Precision(model = "non_filter",hvg_version = "sct",data_version = "count",data = 'pbmc',gene_peak = '500kb', peak_TF = 'motifmatchr',
          directory = "/home/zc354/GRN/output/predicted_network/", true_path ='/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv'):
    
    pred_path=directory+model+"_"+hvg_version+"_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
    if(peak_TF == 'FIMO'):
        pred_path_1=directory+"filter_"+hvg_version+"_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
    else:
        pred_path_1=directory+"filter_sct_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
        
    
    trueEdgesDF = pd.read_csv(true_path,sep = ',', header = 0, index_col = None)
    predEdgesDF = pd.read_csv(pred_path, sep = ',', header =  0, index_col = None)
    predEdgesDF_1 = pd.read_csv(pred_path_1, sep = ',', header =  0, index_col = None)
    
    print('length of pred-network:'+str(len(predEdgesDF['Gene1'])))
    print('length of pred-network TF:'+str(len(set(predEdgesDF['Gene1']))))
    print('length of pred-network gene:'+str(len(set(predEdgesDF['Gene2']))))
    print('length of true-network:'+str(len(trueEdgesDF['Gene1'])))
    
    predEdgesDF.drop(predEdgesDF[predEdgesDF['Edgeweight'] == 0].index,inplace=True)

    trueEdgesDF = trueEdgesDF.loc[(trueEdgesDF['Gene1'] != trueEdgesDF['Gene2'])]
    predEdgesDF = predEdgesDF.loc[(predEdgesDF['Gene1'] != predEdgesDF['Gene2'])]

    trueEdgesDF.drop_duplicates(keep = 'first', inplace=True)
    predEdgesDF.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)

    print('length of true-network TF:'+str(len(set(trueEdgesDF['Gene1']))))
    print('length of true-network gene:'+str(len(set(trueEdgesDF['Gene2']))))
    
    print('length of pred-network TF_filtered:'+str(len(set(predEdgesDF['Gene1']))))
    print('length of pred-network gene_filtered:'+str(len(set(predEdgesDF['Gene2']))))
    
    true_TF = set(trueEdgesDF['Gene1'])
    true_tg = set(trueEdgesDF['Gene2'])

    pred_TF = set(predEdgesDF_1['Gene1'])
    pred_tg = set(predEdgesDF_1['Gene2'])

    TFset = (true_TF & pred_TF) 
    tgset = (true_tg & pred_tg)
    
    print('length of TF_intersect:'+str(len(TFset)))
    print('length of tg_intersect:'+str(len(tgset)))

    trueEdgesDF = trueEdgesDF[(trueEdgesDF['Gene1'].isin(TFset))&(trueEdgesDF['Gene2'].isin(tgset))]
    predEdgesDF = predEdgesDF[(predEdgesDF['Gene1'].isin(TFset))&(predEdgesDF['Gene2'].isin(tgset))]
    
    TrueEdgesDF = pd.DataFrame(columns=['Edge'])
    TrueEdgesDF['Edge'] = trueEdgesDF['Gene1'].map(str) +"|"+ trueEdgesDF['Gene2'].map(str)
    PredEdgesDF = pd.DataFrame(columns=['Edge','Weight'])
    PredEdgesDF['Edge'] = predEdgesDF['Gene1'].map(str) +"|"+ predEdgesDF['Gene2'].map(str)
    PredEdgesDF['Weight'] = predEdgesDF['Edgeweight']
    
    #Generate the set of true/pred/possible edges
    TrueEdgesset=set(TrueEdgesDF['Edge'])
    PredEdgesset=set(PredEdgesDF['Edge'])
    num = len(TrueEdgesset)
    
    PredEdgesDF = PredEdgesDF.sort_values(by=['Weight'],ascending=False)
    edgeweightTopk = PredEdgesDF.iloc[num-1].Weight
    print("weight topk="+str(edgeweightTopk))
    nonZeroMin = np.nanmin(PredEdgesDF.Weight.replace(0, np.nan).values)
    print("weight nonzero="+str(nonZeroMin))
    bestVal = max(nonZeroMin, edgeweightTopk)
    print("weight threshold="+str(bestVal))
    dataset = {}
    newDF = PredEdgesDF.loc[(PredEdgesDF['Weight'] >= bestVal)]
    dataset = set(newDF['Edge'])
    print("Positive_num="+str(len(dataset)))
    
    TP_num=len(dataset & TrueEdgesset)

    print("TP_num="+str(TP_num))
    print("True_num="+str(num))
    early_precision = TP_num/num
    
    return early_precision

In [75]:
early_precision = Early_Precision(model = "non_filter",data_version = "count",data = 'bmmc',gene_peak = 'linkpeaks', peak_TF = 'motifmatchr')

length of pred-network:1043500
length of pred-network TF:649
length of pred-network gene:1608
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:649
length of pred-network gene_filtered:1608
length of TF_intersect:370
length of tg_intersect:919
weight topk=0.01194391
weight nonzero=1.913472e-07
weight threshold=0.01194391
Positive_num=7671
TP_num=383
True_num=7671


In [76]:
model = ['non_filter',"filter","multiply","score"]
data = ["bmmc","pbmc"]
gene_peak = ['linkpeaks','500kb']
peak_TF = ['motifmatchr']

In [77]:
for Data,Gene_peak,Peak_TF,Model in product(data,gene_peak,peak_TF,model):
    early_precision = Early_Precision(model = Model,data_version = "count",data = Data,gene_peak = Gene_peak, peak_TF = Peak_TF)
    str1 = Model+"_"+Data+"_"+Gene_peak+"_"+Peak_TF+"="+str(early_precision)
    print(str1)
    print("--------------------------------------------------------------------")

length of pred-network:1043500
length of pred-network TF:649
length of pred-network gene:1608
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:649
length of pred-network gene_filtered:1608
length of TF_intersect:370
length of tg_intersect:919
weight topk=0.01194391
weight nonzero=1.913472e-07
weight threshold=0.01194391
Positive_num=7671
TP_num=383
True_num=7671
non_filter_bmmc_linkpeaks_motifmatchr=0.049928301394863775
--------------------------------------------------------------------
length of pred-network:277467
length of pred-network TF:649
length of pred-network gene:984
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:649
length of pred-network gene_filtered:984
length of TF_intersect:370
length of tg_intersect:919
weight topk=0.0178208022743999
weight nonzero=4.05260825635788e-06
weight threshold=0.0178208022743999
Po

length of pred-network:1958937
length of pred-network TF:1119
length of pred-network gene:1983
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:961
length of pred-network gene_filtered:1983
length of TF_intersect:665
length of tg_intersect:1766
weight topk=0.01102819
weight nonzero=2.253239e-12
weight threshold=0.01102819
Positive_num=15844
TP_num=750
True_num=15844
multiply_pbmc_500kb_motifmatchr=0.0473365311789952
--------------------------------------------------------------------
length of pred-network:1958942
length of pred-network TF:1120
length of pred-network gene:1983
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:962
length of pred-network gene_filtered:1983
length of TF_intersect:665
length of tg_intersect:1766
weight topk=0.0110432707087576
weight nonzero=1.40663561643137e-12
weight threshold=0.0110432707087576
P

In [79]:
def Early_Precision_2(model = "non_filter",hvg_version = "sct",data_version = "count",data = 'pbmc',gene_peak = '500kb', peak_TF = 'motifmatchr',
          directory = "/home/zc354/GRN/output/predicted_network/", true_path ='/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv'):
    
    pred_path=directory+model+"_"+hvg_version+"_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
    if(peak_TF == 'FIMO'):
        pred_path_1=directory+"filter_"+hvg_version+"_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
    else:
        pred_path_1=directory+"non_filter_sct_"+data_version+"_"+data+"_"+gene_peak+"_"+peak_TF+'.csv'
        
    
    trueEdgesDF = pd.read_csv(true_path,sep = ',', header = 0, index_col = None)
    predEdgesDF = pd.read_csv(pred_path, sep = ',', header =  0, index_col = None)
    predEdgesDF_1 = pd.read_csv(pred_path_1, sep = ',', header =  0, index_col = None)
    
    print('length of pred-network:'+str(len(predEdgesDF['Gene1'])))
    print('length of pred-network TF:'+str(len(set(predEdgesDF['Gene1']))))
    print('length of pred-network gene:'+str(len(set(predEdgesDF['Gene2']))))
    print('length of true-network:'+str(len(trueEdgesDF['Gene1'])))
    
    predEdgesDF.drop(predEdgesDF[predEdgesDF['Edgeweight'] == 0].index,inplace=True)

    trueEdgesDF = trueEdgesDF.loc[(trueEdgesDF['Gene1'] != trueEdgesDF['Gene2'])]
    predEdgesDF = predEdgesDF.loc[(predEdgesDF['Gene1'] != predEdgesDF['Gene2'])]

    trueEdgesDF.drop_duplicates(keep = 'first', inplace=True)
    predEdgesDF.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)

    print('length of true-network TF:'+str(len(set(trueEdgesDF['Gene1']))))
    print('length of true-network gene:'+str(len(set(trueEdgesDF['Gene2']))))
    
    print('length of pred-network TF_filtered:'+str(len(set(predEdgesDF['Gene1']))))
    print('length of pred-network gene_filtered:'+str(len(set(predEdgesDF['Gene2']))))
    
    true_TF = set(trueEdgesDF['Gene1'])
    true_tg = set(trueEdgesDF['Gene2'])

    pred_TF = set(predEdgesDF_1['Gene1'])
    pred_tg = set(predEdgesDF_1['Gene2'])

    TFset = (true_TF & pred_TF) 
    tgset = (true_tg & pred_tg)
    
    print('length of TF_intersect:'+str(len(TFset)))
    print('length of tg_intersect:'+str(len(tgset)))

    trueEdgesDF = trueEdgesDF[(trueEdgesDF['Gene1'].isin(TFset))&(trueEdgesDF['Gene2'].isin(tgset))]
    predEdgesDF = predEdgesDF[(predEdgesDF['Gene1'].isin(TFset))&(predEdgesDF['Gene2'].isin(tgset))]
    
    TrueEdgesDF = pd.DataFrame(columns=['Edge'])
    TrueEdgesDF['Edge'] = trueEdgesDF['Gene1'].map(str) +"|"+ trueEdgesDF['Gene2'].map(str)
    PredEdgesDF = pd.DataFrame(columns=['Edge','Weight'])
    PredEdgesDF['Edge'] = predEdgesDF['Gene1'].map(str) +"|"+ predEdgesDF['Gene2'].map(str)
    PredEdgesDF['Weight'] = predEdgesDF['Edgeweight']
    
    #Generate the set of true/pred/possible edges
    TrueEdgesset=set(TrueEdgesDF['Edge'])
    PredEdgesset=set(PredEdgesDF['Edge'])
    num = len(TrueEdgesset)
    
    PredEdgesDF = PredEdgesDF.sort_values(by=['Weight'],ascending=False)
    edgeweightTopk = PredEdgesDF.iloc[num-1].Weight
    print("weight topk="+str(edgeweightTopk))
    nonZeroMin = np.nanmin(PredEdgesDF.Weight.replace(0, np.nan).values)
    print("weight nonzero="+str(nonZeroMin))
    bestVal = max(nonZeroMin, edgeweightTopk)
    print("weight threshold="+str(bestVal))
    dataset = {}
    newDF = PredEdgesDF.loc[(PredEdgesDF['Weight'] >= bestVal)]
    dataset = set(newDF['Edge'])
    print("Positive_num="+str(len(dataset)))
    
    TP_num=len(dataset & TrueEdgesset)

    print("TP_num="+str(TP_num))
    print("True_num="+str(num))
    early_precision = TP_num/num
    
    return early_precision

In [80]:
for Data,Gene_peak,Peak_TF,Model in product(data,gene_peak,peak_TF,model):
    early_precision = Early_Precision_2(model = Model,data_version = "count",data = Data,gene_peak = Gene_peak, peak_TF = Peak_TF)
    str1 = Model+"_"+Data+"_"+Gene_peak+"_"+Peak_TF+"="+str(early_precision)
    print(str1)
    print("--------------------------------------------------------------------")

length of pred-network:1043500
length of pred-network TF:649
length of pred-network gene:1608
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:649
length of pred-network gene_filtered:1608
length of TF_intersect:370
length of tg_intersect:1440
weight topk=0.01026635
weight nonzero=1.728565e-08
weight threshold=0.01026635
Positive_num=10879
TP_num=513
True_num=10879
non_filter_bmmc_linkpeaks_motifmatchr=0.04715506939976101
--------------------------------------------------------------------
length of pred-network:277467
length of pred-network TF:649
length of pred-network gene:984
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:649
length of pred-network gene_filtered:984
length of TF_intersect:370
length of tg_intersect:1440
weight topk=0.0139732299720357
weight nonzero=4.05260825635788e-06
weight threshold=0.0139732299720357

length of pred-network:1958937
length of pred-network TF:1119
length of pred-network gene:1983
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:961
length of pred-network gene_filtered:1983
length of TF_intersect:665
length of tg_intersect:1775
weight topk=0.0110107483511651
weight nonzero=2.253239e-12
weight threshold=0.0110107483511651
Positive_num=15897
TP_num=751
True_num=15897
multiply_pbmc_500kb_motifmatchr=0.047241617915329936
--------------------------------------------------------------------
length of pred-network:1958942
length of pred-network TF:1120
length of pred-network gene:1983
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:962
length of pred-network gene_filtered:1983
length of TF_intersect:665
length of tg_intersect:1775
weight topk=0.011024398758964
weight nonzero=1.40663561643137e-12
weight threshold=0.0

In [82]:
for Model in model:
    early_precision = Early_Precision(model = Model,data_version = "norm",data = 'pbmc',gene_peak = 'linkpeaks', peak_TF = 'motifmatchr')
    str1 = Model+"_"+Data+"_"+Gene_peak+"_"+Peak_TF+"="+str(early_precision)
    print(str1)
    print("--------------------------------------------------------------------")

length of pred-network:2237882
length of pred-network TF:1119
length of pred-network gene:2000
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:961
length of pred-network gene_filtered:2000
length of TF_intersect:665
length of tg_intersect:1294
weight topk=0.0092554503421022
weight nonzero=2.3102954494097697e-11
weight threshold=0.0092554503421022
Positive_num=12537
TP_num=540
True_num=12537
non_filter_pbmc_500kb_motifmatchr=0.043072505384063174
--------------------------------------------------------------------
length of pred-network:904535
length of pred-network TF:1119
length of pred-network gene:1405
length of true-network:386293
length of true-network TF:2163
length of true-network gene:23378
length of pred-network TF_filtered:961
length of pred-network gene_filtered:1405
length of TF_intersect:665
length of tg_intersect:1294
weight topk=0.01324248
weight nonzero=6.540614e-11
weight threshold=0.01324